In [3]:
import csv
import os
from collections import defaultdict
import random
import sys
sys.path.append('/mnt/c/Users/gweld/sidewalk/sidewalk_ml/')

import GSVutils.utils
from GSVutils.utils import GSV_IMAGE_WIDTH, GSV_IMAGE_HEIGHT
from GSVutils.point import Point

In [4]:
newberg_csv = 'newberg-labels.csv'
seattle_csv = 'seattle-labels.csv'
gt_csv      = '/mnt/c/Users/gweld/sidewalk/sidewalk_ml/ground_truth/ground_truth_labels.csv'

#run_on = (newberg_csv, seattle_csv)
run_on = (gt_csv,)

In [5]:
num_nulls_per_pano = 2
x_min, x_max = 300, GSV_IMAGE_WIDTH - 300
y_min, y_max = -1700, 0
distance_thresh = 700

In [6]:
label_from_int   = ('Curb Cut', 'Missing Cut', 'Obstruction', 'Sfc Problem', '5', '6', '7')

In [7]:
with open(newberg_csv) as f:
    reader = csv.reader(f)
    print next(reader)
    print next(reader)

['gsv_panorama_id', 'sv_image_x', 'sv_image_y', 'label_type_id', 'photagrapher_heading', 'heading', 'is_researcher']
['stxXyCKAbd73DmkM2vsIHA', '9676', '-455', '1', '50.4137916565', '253.375', 't']


In [10]:
photog_headings = dict()
for input_file in run_on:
    with open(input_file) as input_f:
        reader = csv.DictReader(input_f)
        for row in reader:
            pano_id = row['gsv_panorama_id']
            photog_heading = row['photographer_heading'] # if this gives an error, try 'photoagrapher'
            if pano_id not in photog_headings:
                photog_headings[pano_id] = float(photog_heading)

print "Got photographer headings for {} panos.".format(len(photog_headings))

Got photographer headings for 145 panos.


In [13]:
for input_file in run_on:
    output_file, _ = os.path.splitext(input_file)
    output_file += '-null.csv'
    
    with open(input_file) as input_f, open(output_file, 'w') as output_f:
        panos = defaultdict(set)
        feats = 0
        reader = csv.DictReader(input_f)
        
        num_nulls = 0
        
        for row in reader:
            label = int(row['label_type_id'])
            pano_id = row['gsv_panorama_id']
            
            x = int(row['sv_image_x'])
            y = int(row['sv_image_y'])
            
            point = Point(x,y)
            
            if label in (1,2,3,4):
                panos[pano_id].add(point)
                feats += 1
        print "Loaded {} features for {} panos.".format(feats, len(panos))
    
        header = reader.fieldnames
        writer = csv.DictWriter(output_f, header)
        for pano_id, features in panos.items():
            for _ in range(num_nulls_per_pano):
                near_existing_feat = True
                while near_existing_feat:
                    x = random.randint(x_min, x_max)
                    y = random.randint(y_min, y_max)
                    null = Point(x,y)
                    near_existing_feat = False
                    for p in features:
                        if p.dist(null) < distance_thresh:
                            near_existing_feat = True
                            break
                
                row = {}
                row['gsv_panorama_id'] = pano_id
                row['sv_image_x']      = x
                row['sv_image_y']      = y
                row['label_type_id']   = 8
                row['photographer_heading'] = photog_headings[pano_id]
                
                writer.writerow(row)
                num_nulls += 1
    print "wrote {} nulls to {}".format(num_nulls, output_file)
    print ""

Loaded 1163 features for 145 panos.
wrote 290 nulls to /mnt/c/Users/gweld/sidewalk/sidewalk_ml/ground_truth/ground_truth_labels-null.csv

